In [5]:
import pandas as pd

# Carica i file
mediated_df = pd.read_csv('final_mediated_schema.csv')
ground_truth_df = pd.read_csv('data/ground_truth.csv')
matched_df = pd.read_csv('matched_companies_detailed.csv')

# Funzione per controllare se una coppia Ã¨ nella ground truth
def is_match(row, ground_truth):
    left_company = row['company_name_left'].strip().lower()
    right_company = row['company_name_right'].strip().lower()
    
    for _, gt in ground_truth.iterrows():
        a = gt['company_a'].strip().lower()
        b = gt['company_b'].strip().lower()

        if (left_company == a and right_company == b) or (left_company == b and right_company == a):
            return 1
    return 0

# Genera la colonna label
matched_df['label'] = matched_df.apply(lambda row: is_match(row, ground_truth_df), axis=1)

# Salva il CSV finale
output_columns = ['company_name_left', 'industry_left', 'headquarters_city_left',
                  'company_name_right', 'industry_right', 'headquarters_city_right', 'label']

final_df = matched_df[output_columns]
final_df.to_csv('training_data.csv', index=False)

print("CSV per l'addestramento generato con successo: training_data.csv")


C:\Users\luca-\AppData\Local\Temp\ipykernel_28412\1271226506.py:4: DtypeWarning: Columns (2,5,6,7,9,14,16,17,18,19,20) have mixed types. Specify dtype option on import or set low_memory=False.
  mediated_df = pd.read_csv('final_mediated_schema.csv')


CSV per l'addestramento generato con successo: training_data.csv


In [4]:


import pandas as pd

# Carica il dataset
file_path = 'entity-matching-transformer/data/training_data.csv'
df = pd.read_csv(file_path)

# Porta la colonna 'label' al primo posto e rimuove eventuali valori mancanti
cols = ['label'] + [col for col in df.columns if col != 'label']
df = df[cols]
df = df.dropna(subset=['label'])  # Rimuove righe senza etichetta

# Salva il file come TSV senza righe vuote
output_path = 'entity-matching-transformer/data/data/train.tsv'
df.to_csv(output_path, sep='\t', index=False, header=False)

print("Colonne riordinate, righe vuote rimosse e file salvato come train.tsv")


Colonne riordinate, righe vuote rimosse e file salvato come train.tsv


In [6]:
import pandas as pd

# 1. Caricamento dei file
pairwise_file = 'matched_companies_detailed.csv'
groundtruth_file = 'data/ground_truth.csv'

pairwise = pd.read_csv(pairwise_file)
groundtruth = pd.read_csv(groundtruth_file)

# 2. Funzioni di normalizzazione
def normalize_name(name):
    return name.strip().lower() if isinstance(name, str) else str(name).lower()

def normalize_pair(left_name, right_name):
    left = normalize_name(left_name)
    right = normalize_name(right_name)
    return tuple(sorted([left, right]))  # es. ('a', 'b') invece di ('b', 'a')

# 3. Aggiungiamo una colonna 'normalized_pair' a entrambi i DataFrame
pairwise['normalized_pair'] = pairwise.apply(
    lambda row: normalize_pair(row['company_name_left'], row['company_name_right']), axis=1
)

groundtruth['normalized_pair'] = groundtruth.apply(
    lambda row: normalize_pair(row['company_a'], row['company_b']), axis=1
)

# 4. Trasformiamo in set per calcolare l'intersezione
pairwise_set = set(pairwise['normalized_pair'])
groundtruth_set = set(groundtruth['normalized_pair'])

common_pairs = pairwise_set.intersection(groundtruth_set)
print(f"Numero di coppie in intersezione: {len(common_pairs)}")

# 5. Selezioniamo dal pairwise soltanto le righe che corrispondono all'intersezione
intersection_df = pairwise[pairwise['normalized_pair'].isin(common_pairs)]

# 6. Esportiamo il risultato in un nuovo CSV
intersection_df.to_csv('intersection_pairs.csv', index=False)

print("✅ Creato il file 'intersection_pairs.csv' con le coppie in intersezione e i valori di name_sim!")


Numero di coppie in intersezione: 24
✅ Creato il file 'intersection_pairs.csv' con le coppie in intersezione e i valori di name_sim!


In [ ]:
import pandas as pd
import recordlinkage
import time

# Avvio del timer
start_time = time.time()

# 1) Caricamento e pre-elaborazione dati
schema_file = "main_outputs/final_mediated_schema.csv"
df = pd.read_csv(schema_file)

# Seleziona le colonne di interesse
df = df[['company_name', 'industry', 'headquarters_city']]

# Gestione dei valori mancanti: sostituzione con stringa vuota
df.fillna('', inplace=True)

# Normalizza gli attributi: rimozione degli spazi e conversione in minuscolo
df = df.apply(lambda x: x.str.strip().str.lower() if x.dtype == 'object' else x)

# Filtra i record indesiderati: rimuove righe in cui 'company_name' è vuoto o contiene solo '\n'
df = df[~(df['company_name'].str.strip().isin(['', '\n']))]

print(f"Numero di record iniziali: {len(df)}")

# 2) Creazione dell'indice per il blocking: uso di SortedNeighbourhood sul campo 'company_name'
indexer = recordlinkage.Index()
indexer.sortedneighbourhood(left_on=['company_name'], window=5)

# Genera le coppie candidate
candidate_pairs = indexer.index(df)
print(f"Numero di coppie candidate: {len(candidate_pairs)}")

# 3) Definizione delle regole di confronto: per ogni attributo viene applicato il confronto con Jaro-Winkler
compare = recordlinkage.Compare()

for col in df.columns:
    compare.string(col, col, method='jarowinkler', label=f'{col}_similarity')

# 4) Calcolo della matrice di similarità per le coppie candidate
features = compare.compute(candidate_pairs, df)
print(f"Dimensioni della matrice di similarità: {features.shape}")

# 5) Imposta la similarità a zero se, per un determinato attributo, entrambe le celle sono vuote
for col in df.columns:
    empty_cells = (df[col] == '')
    pairs_with_empty = candidate_pairs[
        empty_cells.loc[candidate_pairs.get_level_values(0)].values &
        empty_cells.loc[candidate_pairs.get_level_values(1)].values
    ]
    features.loc[pairs_with_empty, f'{col}_similarity'] = 0

# 6) Filtraggio delle coppie candidate in base a regole di similarità:
#    - 'company_name_similarity' deve essere > 0.7
#    - Almeno 2 attributi devono avere una similarità >= soglia (0.75)
threshold = 0.75
matches = features[
    (features['company_name_similarity'] < 0.9) & (features['company_name_similarity'] > 0.7) 
    #((features >= threshold).sum(axis=1) >= 2)
]

print(f"Numero di coppie finali considerate 'match': {len(matches)}")

# 7) Unione con il dataframe originale per ottenere i record affiancati

# a) Resetta l'indice (la MultiIndex diventa 'id_left' e 'id_right')
matches = matches.reset_index()
matches.rename(columns={'level_0': 'id_left', 'level_1': 'id_right'}, inplace=True)

# b) Unisci i dati del record "di sinistra" (id_left)
matches = matches.merge(df, left_on='id_left', right_index=True, how='left', suffixes=('', '_left'))

# c) Unisci i dati del record "di destra" (id_right)
matches = matches.merge(df, left_on='id_right', right_index=True, how='left', suffixes=('_left', '_right'))

# (Opzionale) Elimina le coppie in cui il company_name è identico
matches = matches[matches['company_name_left'] != matches['company_name_right']]

# Salvataggio su file CSV
matches.to_csv("matched_companies_detailed.csv", index=False)
print("✅ File 'matched_companies_detailed.csv' generato con i record affiancati!")

# Calcola e stampa il tempo di esecuzione
end_time = time.time()
execution_time = end_time - start_time
print(f"Tempo di esecuzione: {execution_time:.2f} secondi")


In [ ]:
import pandas as pd
import recordlinkage
from difflib import SequenceMatcher
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, classification_report
import numpy as np
import matplotlib.pyplot as plt

##############################################
# 1. Caricamento e pre-elaborazione dati     #
##############################################

# Caricamento dello schema mediato
schema_file = "main_outputs/final_mediated_schema.csv"
df_schema = pd.read_csv(schema_file)
df_schema = df_schema[['company_name']].copy()
df_schema.dropna(subset=['company_name'], inplace=True)
df_schema['company_name'] = df_schema['company_name'].str.strip().str.lower()

# Creiamo una nuova colonna con il primo carattere per il blocking
df_schema['first_letter'] = df_schema['company_name'].str[0]

print(f"Numero di record nello schema mediato: {len(df_schema)}")

# Caricamento della Ground Truth
gt_file = "data/ground_truth.csv"
df_gt = pd.read_csv(gt_file)
for col in ['company_a', 'company_b']:
    df_gt[col] = df_gt[col].fillna("").str.strip().str.lower()

# Creiamo una chiave ordinata per ciascuna coppia della GT
df_gt['pair'] = df_gt.apply(lambda row: tuple(sorted([row['company_a'], row['company_b']])), axis=1)
gt_pairs = set(df_gt['pair'])
print(f"Numero di coppie nella Ground Truth: {len(gt_pairs)}")

##############################################
# 2. Generazione delle coppie candidate (blocking)  #
##############################################

# Usiamo un blocking sul primo carattere del company_name per ridurre il numero di coppie candidate
indexer = recordlinkage.Index()
indexer.block(left_on='first_letter')
candidate_pairs = indexer.index(df_schema)
print(f"Numero di coppie candidate generate (blocking): {len(candidate_pairs)}")

##############################################
# 3. Calcolo delle feature di similarità       #
##############################################

# Calcoliamo la similarità jarowinkler con recordlinkage
compare = recordlinkage.Compare()
compare.string('company_name', 'company_name', method='jarowinkler', label='jarowinkler')
features_df = compare.compute(candidate_pairs, df_schema)
features_df = features_df.reset_index()  # colonne: level_0, level_1, jarowinkler

# Aggiungiamo la feature "seqsim" calcolata con SequenceMatcher
def seq_similarity(idx_left, idx_right):
    name_left = df_schema.loc[idx_left, 'company_name']
    name_right = df_schema.loc[idx_right, 'company_name']
    return SequenceMatcher(None, name_left, name_right).ratio()

features_df['seqsim'] = features_df.apply(lambda row: seq_similarity(row['level_0'], row['level_1']), axis=1)

##############################################
# 4. Costruzione del dataset di training      #
##############################################

# Creiamo una chiave per ciascuna coppia candidata
features_df['pair'] = features_df.apply(lambda row: tuple(sorted([
    df_schema.loc[row['level_0'], 'company_name'],
    df_schema.loc[row['level_1'], 'company_name']
])), axis=1)

# Etichettiamo: 1 se la coppia è presente nella GT, 0 altrimenti
features_df['label'] = features_df['pair'].apply(lambda p: 1 if p in gt_pairs else 0)

print("Distribuzione delle etichette candidate:")
print(features_df['label'].value_counts())

# Campioniamo i negativi per evitare uno sbilanciamento eccessivo
positives = features_df[features_df['label'] == 1]
negatives = features_df[features_df['label'] == 0]
print(f"Positivi: {len(positives)}, Negativi totali: {len(negatives)}")

n_sample_neg = min(len(negatives), len(positives) * 5)
negatives_sample = negatives.sample(n=n_sample_neg, random_state=42)

df_train = pd.concat([positives, negatives_sample], axis=0).reset_index(drop=True)
X = df_train[['jarowinkler', 'seqsim']]
y = df_train['label']

##############################################
# 5. Addestramento del classificatore          #
##############################################

# Dividiamo in training e validation
X_train, X_val, y_train, y_val = train_test_split(X, y, stratify=y, random_state=42)

clf = LogisticRegression()
clf.fit(X_train, y_train)

# Calcoliamo le probabilità sul validation set
y_val_probs = clf.predict_proba(X_val)[:, 1]

# Ottimizziamo la soglia per massimizzare l'F1 score sul validation set
thresholds = np.linspace(0, 1, 101)
f1_scores = []
for thresh in thresholds:
    y_pred_thresh = (y_val_probs >= thresh).astype(int)
    f1_scores.append(f1_score(y_val, y_pred_thresh))

best_thresh = thresholds[np.argmax(f1_scores)]
print("\nReport di classificazione (con soglia ottimizzata sul validation set):")
y_pred_opt = (y_val_probs >= best_thresh).astype(int)
print(classification_report(y_val, y_pred_opt))
print(f"Soglia ottimizzata: {best_thresh:.2f}")

# Visualizziamo la distribuzione delle probabilità di match
plt.hist(clf.predict_proba(X)[:, 1], bins=50)
plt.xlabel("Probabilità di match")
plt.ylabel("Frequenza")
plt.title("Distribuzione delle probabilità di match nel training set")
plt.show()

##############################################
# 6. Applicazione del modello a tutte le coppie  #
##############################################

X_all = features_df[['jarowinkler', 'seqsim']]
features_df['match_prob'] = clf.predict_proba(X_all)[:, 1]

# Invece di usare direttamente la soglia ottimizzata, possiamo impostare una soglia più stringente per ridurre i match.
manual_thresh = 0.75
df_filtered = features_df[features_df['match_prob'] >= manual_thresh].copy()
print(f"\nNumero di coppie filtrate con soglia manuale {manual_thresh}: {len(df_filtered)}")

##############################################
# 7. Salvataggio del risultato               #
##############################################

# Uniamo i dettagli dello schema mediato alle coppie filtrate
df_filtered = df_filtered.merge(df_schema[['company_name']], left_on='level_0', right_index=True, how='left', suffixes=('', '_left'))
df_filtered = df_filtered.merge(df_schema[['company_name']], left_on='level_1', right_index=True, how='left', suffixes=('_left', '_right'))

output_file = "matched_companies_filtered_gt.csv"
df_filtered.to_csv(output_file, index=False)
print(f"\n✅ File '{output_file}' generato con le coppie che il modello considera come match, utilizzando blocking e soglia {manual_thresh}!")


C:\Users\luca-\AppData\Local\Temp\ipykernel_28412\1859461966.py:15: DtypeWarning: Columns (2,5,6,7,9,14,16,17,18,19,20) have mixed types. Specify dtype option on import or set low_memory=False.
  df_schema = pd.read_csv(schema_file)


Numero di record nello schema mediato: 75793
Numero di coppie nella Ground Truth: 205
Numero di coppie candidate generate: 201353
Distribuzione delle etichette candidate:
label
0    200959
1       394
Name: count, dtype: int64
Positivi: 394, Negativi totali: 200959

Report di classificazione (soglia ottimizzata):
              precision    recall  f1-score   support

           0       0.98      0.59      0.74       493
           1       0.31      0.94      0.47        98

    accuracy                           0.65       591
   macro avg       0.65      0.76      0.60       591
weighted avg       0.87      0.65      0.69       591

Soglia ottimizzata: 0.20

Numero di coppie filtrate dal modello con soglia 0.20: 78633

✅ File 'matched_companies_filtered_gt.csv' generato con solo le coppie che il modello considera come match (simili alla GT)!
